In [59]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [60]:
def find_nan(df):
    nan_list = []
    ls = df.isna().any()
    ls = ls.to_dict()
    for a in ls:
        if ls[a] is True:
            nan_list.append(a)
    return nan_list

In [61]:
def fill_mean(df,nanval):
    diffs = []
    means = []
    for column in nanval:
        diffs = df[column].unique().tolist()
        diffs = [x for x in diffs if str(x) != 'nan']
        maxnum = df[column].value_counts()[diffs[0]]
        maxval = diffs[0]
        for val in diffs:
            if maxnum < df[column].value_counts()[val]:
                maxnum = df[column].value_counts()[val]
                maxval = val
        df[column].fillna(maxval,inplace=True)
        means.append(maxval)
    
    return df , means

In [62]:
df_train = pd.read_csv('kkbox-music-recommendation-challenge/train.csv',index_col=False)

In [63]:
df_members = pd.read_csv('kkbox-music-recommendation-challenge/members.csv',index_col=False)

In [64]:
df_songs = pd.read_csv('kkbox-music-recommendation-challenge/songs.csv',index_col=False)

In [65]:
df_extra = pd.read_csv('kkbox-music-recommendation-challenge/song_extra_info.csv',index_col=False)

In [66]:
df_test = pd.read_csv('kkbox-music-recommendation-challenge/test.csv',index_col=False)

In [67]:
nan_train = find_nan(df_train)
df_train , train_na = fill_mean(df_train,nan_train)

In [68]:
df_test['source_system_tab'].fillna(train_na[0],inplace=True)
df_test['source_screen_name'].fillna(train_na[1],inplace=True)
df_test['source_type'].fillna(train_na[2],inplace=True)

In [69]:
train_song_mer = df_train.merge(df_songs,on='song_id',how='left')
test_song_mer = df_test.merge(df_songs,on='song_id',how='left')

In [70]:
tr_so_mm_mer = train_song_mer.merge(df_members,on='msno',how='left')
te_so_mm_mer = test_song_mer.merge(df_members,on='msno',how='left')

In [71]:
train_final = tr_so_mm_mer.merge(df_extra,on='song_id',how='left')
test_final = te_so_mm_mer.merge(df_extra,on='song_id',how='left')

In [72]:
train_final['composer'].fillna('unknown_comp',inplace=True)
train_final['lyricist'].fillna('unknown_lyrc',inplace=True)
train_final['artist_name'].fillna('unknown_art',inplace=True)
train_final['name'].fillna('unknown_name',inplace=True)
train_final['isrc'].fillna('unknown_isrc',inplace=True)
train_final['genre_ids'].fillna('465',inplace=True)
test_final['composer'].fillna('unknown_comp',inplace=True)
test_final['lyricist'].fillna('unknown_lyrc',inplace=True)
test_final['artist_name'].fillna('unknown_art',inplace=True)
test_final['name'].fillna('unknown_name',inplace=True)
test_final['isrc'].fillna('unknown_isrc',inplace=True)
test_final['genre_ids'].fillna('465',inplace=True)

In [73]:
son_len = train_final['song_length'].tolist()
son_len = [x for x in son_len if str(x) != 'nan']
avg_son = int(sum(son_len)/len(son_len))
train_final['song_length'].fillna(avg_son,inplace=True)
test_final['song_length'].fillna(avg_son,inplace=True)

In [74]:
final_nan = find_nan(train_final)
test_nan = find_nan(test_final)

In [75]:
train_final, train_na = fill_mean(train_final,final_nan)
test_final['language'].fillna(train_na[0],inplace=True)
test_final['gender'].fillna(train_na[1],inplace=True)

In [76]:
obj_col= train_final.columns.tolist()
for col in obj_col:
    if train_final[col].dtype == 'object':
        train_final[col] = train_final[col].astype('category')

In [77]:
obj_col= test_final.columns.tolist()
for col in obj_col:
    if test_final[col].dtype == 'object':
        test_final[col] = test_final[col].astype('category')

In [78]:
train_final = pd.concat([train_final, pd.get_dummies(train_final['genre_ids'], drop_first = True)], axis = 1)
train_final.drop('genre_ids', axis = 1, inplace = True)

In [79]:
end_columns = train_final.columns.tolist()
gen_end = []
for col in end_columns:
    if train_final[col].dtype == 'uint8':
        gen_end.append(col)

In [ ]:
diff_gens_train = []
for col in gen_end:
    val = [x.strip() for x in col.split("|")]
    for v in val:
        diff_gens_train.append(v)
        diff_gens_train = list(set(diff_gens_train))

In [ ]:
train_end = train_final
done_gens = []
for col in gen_end:
    if col in diff_gens_train:
        pass
    else:
        gen_split = [x.strip() for x in col.split("|")]
        for gen in gen_split:
            if gen in gen_end:
                train_end[gen] = train_end[col]+train_end[gen]
                print(gen)
            else:
                train_end[gen] = train_end[col]
                print(gen)
        del train_end[col]

1011
2189
367
1011
359
1011
691
1040
1155
1096
958
109
1138
109
118
109
1259
109
139
109
2122
109
359
109
465
109
921
109
94
1138
2122
1145
1103


In [ ]:
test_back = test_final
test_final = pd.concat([test_final, pd.get_dummies(test_final['genre_ids'], drop_first = True)], axis = 1)
test_final.drop('genre_ids', axis = 1, inplace = True)

In [ ]:
end_columns = test_final.columns.tolist()
print(len(end_columns))
gen_test = []
for col in end_columns:
    if test_final[col].dtype == 'uint8':
        gen_test.append(col)
        print(col)

In [ ]:
diff_gens_test = []
for col in gen_test:
    val = [x.strip() for x in col.split("|")]
    for v in val:
        diff_gens_test.append(v)
        diff_gens_test = list(set(diff_gens_test))

In [ ]:
end_columns = test_final.columns.tolist()
for col in gen_test:
    if col in diff_gens_test:
        pass
    else:
        val = [x.strip() for x in col.split("|")]
        for v in val:
            if v in end_columns:
                test_final[v] = test_final[col]+test_final[v]
                print(v)
            else:
                test_final[v] = test_final[col]
                print(v)
        del test_final[col]

In [ ]:
tr_co = train_end.columns.tolist()
te_co = test_final.columns.tolist()
left = []
for col in tr_co:
    if col in te_co:
        pass
    else:
        left.append(col)
        print(col)
right = []
for col in te_co:
    if col in tr_co:
        pass
    else:
        right.append(col)
        print(col)

In [ ]:
for col in left:
    if col!='target':
        test_final[col] = 0
        test_final[col] = test_final[col].astype('uint8')
for col in right:
    if col!='id':
        train_end[col] = 0
        train_end[col] = train_end[col].astype('uint8')

In [ ]:
len(train_end.columns.tolist())

In [ ]:
len(test_final.columns.tolist())

In [ ]:
train_column = train_end.columns.tolist()
test_column = test_final.columns.tolist()

In [ ]:
gens = []
for col in train_column:
    if train_end[col].dtype == 'uint8':
        gens.append(col)
for col in test_column:
    if test_final[col].dtype == 'uint8':
        gens.append(col)


In [ ]:
gen_count = {}
for col in gens:
    temp = int(sum(train_end[col]))+int(sum(test_final[col]))
    gen_count[col] = temp
    

In [ ]:
mini = gen_count["465"]
maxi = gen_count["465"]
avg = 0
count = 0
for col in gen_count:
    if gen_count[col]<mini:
        mini = gen_count[col]
    if gen_count[col]>maxi:
        maxi = gen_count[col]
    avg = avg + gen_count[col]
    count+=1
avg = int(avg/count)
print(maxi)
print(mini)
print(avg)

In [40]:
avg_big = 0
for col in gen_count:
    if gen_count[col]<avg:
        avg_big+=1
print(avg_big)
print(count)

156
171


In [41]:
navg = 0
ncount = 0
for col in gen_count:
    if gen_count[col]<avg:
        navg = navg + gen_count[col]
        ncount+=1
navg = int(navg/ncount)
print(navg)

3285


In [44]:
s_100 = 0
for col in gen_count:
    if gen_count[col]<10:
        s_100+=1
print(s_100)
print(count)

25
171


In [45]:
for col in gen_count:
    if gen_count[col]<10:
        del train_end[col]
        del test_final[col]
        print(col)

1124
1131
1187
1194
1201
152
1579
2150
2183
2192
2245
331
338
677
779
87
125
310
2008
296
1061
1089
303
765
2045


In [58]:

# train_end = train_end.drop(['bd'], axis=1)
# test_final = test_final.drop(['bd'], axis=1)
X_train = train_end.drop(['target'], axis=1)
y_train = train_end['target'].values


X_test = test_final.drop(['id'], axis=1)
ids = test_final['id'].values

d_train_final = lgb.Dataset(X_train, y_train)
watchlist_final = lgb.Dataset(X_train, y_train)

KeyboardInterrupt: 

In [54]:
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'dart',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc'
    }

%time model_f1 = lgb.train(params, train_set=d_train_final,  valid_sets=watchlist_final, verbose_eval=5)

/usr/local/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[5]	valid_0's auc: 0.712192
[10]	valid_0's auc: 0.725438
[15]	valid_0's auc: 0.735081
[20]	valid_0's auc: 0.741172
[25]	valid_0's auc: 0.745467
[30]	valid_0's auc: 0.74812
[35]	valid_0's auc: 0.750722
[40]	valid_0's auc: 0.752724
[45]	valid_0's auc: 0.754278
[50]	valid_0's auc: 0.755592
[55]	valid_0's auc: 0.757481
[60]	valid_0's auc: 0.758913
[65]	valid_0's auc: 0.761195
[70]	valid_0's auc: 0.763108
[75]	valid_0's auc: 0.764076
[80]	valid_0's auc: 0.765578
[85]	valid_0's auc: 0.765473
[90]	valid_0's auc: 0.766038
[95]	valid_0's auc: 0.766645
[100]	valid_0's auc: 0.767772
[105]	valid_0's auc: 0.768009
[110]	valid_0's auc: 0.7681
[115]	valid_0's auc: 0.770722
[120]	valid_0's auc: 0.771519
[125]	valid_0's auc: 0.771667
[130]	valid_0's auc: 0.771871
[135]	valid_0's auc: 0.772701
[140]	valid_0's auc: 0.773674
[145]	valid_0's auc: 0.77367
[150]	valid_0's auc: 0.774298
[155]	valid_0's auc: 0.774896
[160]	valid_0's auc: 0.774285
[165]	valid_0's auc: 0.775789
[170]	valid_0's auc: 0.776937
[175

In [55]:
p_test_1 = model_f1.predict(X_test)
subm = pd.DataFrame()
subm['id'] = ids 
subm['target'] = p_test_1
subm.to_csv('submission_lgbm_avg.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')

In [56]:
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc'
    }

%time model_f1 = lgb.train(params, train_set=d_train_final,  valid_sets=watchlist_final, verbose_eval=5)

/usr/local/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[5]	valid_0's auc: 0.712192
[10]	valid_0's auc: 0.726979
[15]	valid_0's auc: 0.735093
[20]	valid_0's auc: 0.741074
[25]	valid_0's auc: 0.746046
[30]	valid_0's auc: 0.74961
[35]	valid_0's auc: 0.752731
[40]	valid_0's auc: 0.755675
[45]	valid_0's auc: 0.758321
[50]	valid_0's auc: 0.761164
[55]	valid_0's auc: 0.763384
[60]	valid_0's auc: 0.765289
[65]	valid_0's auc: 0.767156
[70]	valid_0's auc: 0.769129
[75]	valid_0's auc: 0.771277
[80]	valid_0's auc: 0.773324
[85]	valid_0's auc: 0.774955
[90]	valid_0's auc: 0.776679
[95]	valid_0's auc: 0.777894


KeyboardInterrupt: 

In [57]:
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'dart',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc'
    }

%time model_f2 = lgb.train(params, train_set=d_train_final,  valid_sets=watchlist_final, verbose_eval=5)

/usr/local/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[5]	valid_0's auc: 0.712192
[10]	valid_0's auc: 0.725438
[15]	valid_0's auc: 0.735081
[20]	valid_0's auc: 0.741172
[25]	valid_0's auc: 0.745467
[30]	valid_0's auc: 0.74812
[35]	valid_0's auc: 0.750722


KeyboardInterrupt: 

In [ ]:
print('Making predictions')
p_test_1 = model_f1.predict(X_test)
p_test_2 = model_f2.predict(X_test)
p_test_avg = np.mean([p_test_1, p_test_2], axis = 0)


print('Done making predictions')